In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-It" , groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020DA1F81CF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020DA1FB0AC0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
# Step 1: Generate questions and ideal answers for subjects using an LLM

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json


In [4]:
# Define subjects (blanks for now)
subjects = ["Python", "Data Structures", "Algorithms"]

# Prompt to generate technical questions for a subject
question_gen_prompt = PromptTemplate.from_template("""
You are an expert technical interviewer.
Generate 3 technical interview questions and their ideal answers for the subject: {subject}.
Return the output in strict JSON format as:
[
  {{"question": "...", "ideal_answer": "..."}},
  {{"question": "...", "ideal_answer": "..."}},
  {{"question": "...", "ideal_answer": "..."}}
]
""")

# Prompt for evaluating user responses
feedback_prompt = PromptTemplate.from_template("""
You are a technical interviewer. Evaluate the candidate's answer.
Question: {question}
Ideal Answer: {ideal_answer}
Candidate's Answer: {user_answer}

Provide a short and polite evaluation of the answer. Say whether it's correct, partially correct, or incorrect, and suggest how it can be improved (if needed).
""")


# LLM chain for generating questions
question_chain = LLMChain(llm=llm, prompt=question_gen_prompt)
feedback_chain = LLMChain(llm=llm, prompt=feedback_prompt)

C:\Users\SYAM\AppData\Local\Temp\ipykernel_8432\1954802701.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  question_chain = LLMChain(llm=llm, prompt=question_gen_prompt)


In [8]:
def extract_json_from_text(raw_text: str) -> dict:
    """
    Extracts the substring between the first '{' and the last '}' in the text.
    Attempts to parse it as JSON and return the object.
    If parsing fails, returns an empty dictionary.
    """
    start = raw_text.find('[')
    end = raw_text.rfind(']')
    
    if start != -1 and end != -1 and end > start:
        json_str = raw_text[start:end + 1]
        try:
            parsed = json.loads(json_str)
            return parsed
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}. Raw text: {raw_text}")
            return {}
    else:
        print("No valid JSON found in the text.")
    return {}

def generate_questions_for_subject(subject):
    print(f"\nGenerating questions for subject: {subject}")
    raw_output = question_chain.run({"subject": subject})
    try:
        return extract_json_from_text(raw_output)
    except json.JSONDecodeError:
        print("Failed to parse JSON. Raw Output:", raw_output)
        return []


In [9]:
def interview_user(subject, questions):
    print(f"\n--- Interview: {subject} ---")
    for i, q in enumerate(questions, 1):
        print(f"\nQuestion {i}: {q['question']}")
        user_answer = input("Your Answer: ")
        evaluation = feedback_chain.run({
            "question": q["question"],
            "ideal_answer": q["ideal_answer"],
            "user_answer": user_answer
        })
        print("Feedback:", evaluation)

In [10]:
print("\n=== Technical Interview Simulator ===")
for subject in subjects:
    questions = generate_questions_for_subject(subject)
    interview_user(subject, questions)
    input("\nPress Enter to continue to next subject...")


=== Technical Interview Simulator ===

Generating questions for subject: Python

--- Interview: Python ---

Question 1: Explain the difference between a list and a tuple in Python, and provide examples of when you might use each.
Feedback: The candidate's answer is partially correct. 

**Strengths:**

* They correctly identify the key difference between lists and tuples: mutability.
* They provide valid use cases for both lists and tuples.
* The code examples are clear and demonstrate the concepts well.

**Areas for Improvement:**

*  While mentioning performance, the candidate doesn't elaborate on how tuples are generally faster due to their immutability. This is a significant point of distinction.
*  The explanation could be more concise and structured.  

**Suggested Improvement:**

"Your answer is on the right track! You accurately described the mutability difference and provided good examples. To further strengthen your explanation, you could  highlight the performance advantage 